In [10]:
import pandas as pd
import numpy as np

# STEP 1: Load your dataset (replace with actual path or query)
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
# STEP 3: Aggregate to hashtag-day level
df['post_date'] = df['createTimeISO'].dt.date
import pandas as pd

# 1. Ensure datetime format
df['post_date'] = pd.to_datetime(df['post_date'])

# 2. Limit to the last 2 full months
latest_date = df['post_date'].max()
end_of_last_month = latest_date.replace(day=1)
start_of_two_months_ago = end_of_last_month - pd.DateOffset(months=2)

df_recent = df[df['post_date'] >= start_of_two_months_ago]

# 3. Extract month
df_recent['month'] = df_recent['post_date'].dt.to_period('M')

# 4. Count hashtags per month
hashtag_counts = df_recent.groupby(['hashtag_name', 'month']).size().unstack(fill_value=0)

# 5. Get last 2 months
prev_month = hashtag_counts.columns[-2]
last_month = hashtag_counts.columns[-1]

# 6. Filter to hashtags:
# - At least 50 posts this month
# - And growing vs previous month
# Require at least 10 mentions in the previous month too
filtered = hashtag_counts[
    (hashtag_counts[last_month] >= 5) &
    (hashtag_counts[prev_month] >= 5) &  # Added
    (hashtag_counts[last_month] > hashtag_counts[prev_month])
]


/var/folders/5k/b8_r8jw502n4zztm42zd76m80000gn/T/ipykernel_40780/1005129917.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recent['month'] = df_recent['post_date'].dt.to_period('M')


In [11]:

# 7. Calculate growth rate
filtered['growth_rate'] = (
    (filtered[last_month] - filtered[prev_month]) / 
    filtered[prev_month].replace(0, 1)
)

# 8. Select top 10 by growth rate
top10_hashtags = filtered.sort_values('growth_rate', ascending=False).head(10)

# 9. Display
print(top10_hashtags[[prev_month, last_month, 'growth_rate']])


month              2025-05  2025-06  growth_rate
hashtag_name                                    
skincarethatworks        5       17     2.400000
veganskincare            6       16     1.666667
luxuryskincare           5       13     1.600000
cerave                   5       13     1.600000
dmhaul                   5       13     1.600000
arencia                  5       12     1.400000
foru                     5       11     1.200000
abendroutine             7       14     1.000000
tiktokshop              10       19     0.900000
veganbeauty              8       15     0.875000


/var/folders/5k/b8_r8jw502n4zztm42zd76m80000gn/T/ipykernel_40780/1464080539.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['growth_rate'] = (


In [16]:
import pandas as pd

# STEP 1: Load your dataset (replace with actual path or query)
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
# STEP 3: Aggregate to hashtag-day level
df['post_date'] = df['createTimeISO'].dt.date

# 1. Ensure datetime format
df['post_date'] = pd.to_datetime(df['post_date'])

# 2. Limit to last 2 full months
latest_date = df['post_date'].max()
end_of_last_month = latest_date.replace(day=1)
start_of_two_months_ago = end_of_last_month - pd.DateOffset(months=2)

df_recent = df[df['post_date'] >= start_of_two_months_ago].copy()

# 3. Add month column
df_recent['month'] = df_recent['post_date'].dt.to_period('M')

# 4. Count hashtags per month
hashtag_counts = df_recent.groupby(['hashtag_name', 'month']).size().unstack(fill_value=0)

# 5. Get last two months
prev_month = hashtag_counts.columns[-2]
last_month = hashtag_counts.columns[-1]

# 6. Filter: ≥10 mentions in previous month, ≥50 in current month, positive growth
filtered = hashtag_counts[
    (hashtag_counts[prev_month] >= 10) &
    (hashtag_counts[last_month] >= 10) 
   # (hashtag_counts[last_month] > hashtag_counts[prev_month])
].copy()

# 7. Calculate growth rate
filtered['growth_rate'] = (
    (filtered[last_month] - filtered[prev_month]) / 
    filtered[prev_month].replace(0, 1)
)

# 8. Select top 10 by growth rate
top20_hashtags = filtered.sort_values('growth_rate', ascending=False).head(20)

# 9. Extract list of top hashtags
top_hashtags = top20_hashtags.index.tolist()

# 10. Get example posts from latest month only
example_posts = (
    df[
        (df['hashtag_name'].isin(top_hashtags)) &
        (df['post_date'].dt.to_period('M') == last_month)
    ]
    .sort_values('diggCount', ascending=False)  # or use 'shareCount' etc.
    .groupby('hashtag_name')
    .agg({
        'text': 'first',
        'webVideoUrl': 'first',
        'diggCount': 'first'
    })
)

# 11. Join with top 10
final = top10_hashtags.join(example_posts)

# 12. Optional: clean column names and display
final = final.rename(columns={
    prev_month: f'mentions_{prev_month}',
    last_month: f'mentions_{last_month}',
    'text': 'example_caption',
    'webVideoUrl': 'video_link',
    'diggCount': 'likes'
})

# Show final result
pd.set_option('display.max_colwidth', None)
print(final[['mentions_' + str(prev_month), 'mentions_' + str(last_month), 'growth_rate', 'example_caption', 'video_link']])


                       mentions_2025-05  mentions_2025-06  growth_rate  \
hashtag_name                                                             
tiktokshop                           10                19     0.900000   
melanin                              11                20     0.818182   
drogerie                             10                15     0.500000   
hautgesundheit                       23                34     0.478261   
fypシ                                 17                23     0.352941   
doublecleansing                      13                17     0.307692   
vitaminc                             10                13     0.300000   
koreanskincareroutine                10                13     0.300000   
feuchtigkeitspflege                  14                18     0.285714   
centella                             15                19     0.266667   

                                                                                                               

In [17]:
# Create final result DataFrame
result_df = final[[
    f'mentions_{prev_month}', 
    f'mentions_{last_month}', 
    'growth_rate', 
    'example_caption', 
    'video_link'
]].reset_index()  # resets the index so 'hashtag_name' becomes a column


In [18]:
result_df

,hashtag_name,mentions_2025-05,mentions_2025-06,growth_rate,example_caption,video_link
0,tiktokshop,10,19,0.900000,Ich liebe diese Glas Skin Maske von @svenja.walberg 🩷 schon beim ersten Mal hat die skincare Maske mich sehr überzeugt! Sichert euch jetzt die Maske auf #tiktokshopdeutschland #tiktokmademebuyit #tiktokshop #skincare #maske #svenjawalberg #hautpflege,https://www.tiktok.com/@melisabaniz/video/7512192631254945046
1,melanin,11,20,0.818182,"🎯 Vitiligo? Hol dir deine Haut zurück – natürlich. Weißflecken satt? Die Fivfivgo® VitiCalm Bienengift-Creme ist dein Gamechanger! 💥 Dermatologisch bestätigt, sanft & effektiv – ganz ohne aggressive Behandlungen. 🐝 Mit Bienenvenom + pflanzlichen Wirkstoffen ✅ Regt die Melaninproduktion an (ohne Kortison!) ✅ Beruhigt das Immunsystem ✅ Sichtbare Ergebnisse in 7–10 Tagen ✅ Spendet Feuchtigkeit & gleicht den Hautton aus Tausende Nutzer feiern ihre Haut-Rückkehr. 📲 Jetzt tippen & deine Hautreise starten! #Vitiligo#Melanin#Hautpflege#NatürlichHeilen",https://www.tiktok.com/@peachyxchic_official/video/7512315428853533974
2,drogerie,10,15,0.500000,"Ich kann das alles wirklich nicht mehr, gefärbte vaseline für 19€, ich drehe durch 😭 #skincare #hautpflege #drogerie #fyp #viral",https://www.tiktok.com/@xskincare/video/7513510848074108182
3,hautgesundheit,23,34,0.478261,"🌊💧 Warum spritziges, kohlensäurehaltiges Wasser nicht die beste Wahl für deine Haut ist! 💧🌊 🌟 Heute wollen wir über etwas sprechen, das vielen von uns am Herzen liegt: unsere Haut, insbesondere bei Akne. Viele von uns greifen gerne zu spritzigem, kohlensäurehaltigem Wasser – erfrischend und lecker, oder? Aber ist es wirklich so gut für unsere Haut? 👉 Hier sind einige Gründe, warum kohlensäurehaltiges Wasser nicht ideal für Akne sein könnte: 1. Säuregehalt: Kohlensäurehaltiges Wasser hat einen niedrigeren pH-Wert und ist damit saurer. Ein zu saurer pH-Wert kann die Hautbarriere angreifen und zu Reizungen führen, besonders bei empfindlicher Haut. 2. Entzündungen: Die enthaltenen Kohlensäureblasen können bei manchen Menschen ein Hautempfinden von „Prickeln“ oder „Stechen“ auslösen, das sich negativ auf entzündliche Hautzustände wie Akne auswirken kann. 3. Dehydrierung: Obwohl spritziges Wasser erfrischend sein kann, kann der hohe Kohlensäuregehalt dazu führen, dass einige Menschen weniger durstig sind und dadurch die empfohlene Tagesmenge an Flüssigkeit nicht erreichen. Eine gute Hydration ist jedoch entscheidend für eine gesunde Haut! 💡 Tipps für deine Hautpflege: Anstatt zu kohlensäurehaltigem Wasser zu greifen, versuche es mit stillem Wasser oder Tees! Diese Optionen sind nicht nur sanfter zu deiner Haut, sondern unterstützen auch die optimale Hydratation. Was trinkt ihr am liebsten? Lasst es uns in den Kommentaren wissen! 👇💬 Bleibt gesund und sorgt gut für eure Haut! 🌟💖 #Hautpflege #Akne #Kohlensäure #Hautgesundheit #StayHydrated #BeautyTips Aknebehandlung, Hautpflege, Kohlensäurehaltiges Wasser, Sprudelwasser und Hautgesundheit, Hautirritation, pH-Wert der Haut, Hautpflege-Tipps, Kohlenstoffdioxid und Haut, Säure in Getränken, Hautfeuchtigkeit, Stilles Wasser vs. Sprudelwasser, Entzündungshemmende Maßnahmen, Gesunde Haut, Hauttyp und Ernährung, Hydratation und Akne, Kohlensäure und Hautempfindlichkeit, Ernährung für klare Haut, Natürliche Hautpflege, Kohlensäurehaltige Getränke und Gesundheit, Akne und Ernährung.",https://www.tiktok.com/@medicalcosmeticnina/video/7512770112462654743
4,fypシ,17,23,0.352941,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950
5,doublecleansing,13,17,0.307692,ad A cleansing wipe

In [32]:
import pandas as pd

# STEP 1: Load your dataset (replace with actual path or query)
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
# STEP 3: Aggregate to hashtag-day level
df['post_date'] = df['createTimeISO'].dt.date

# 1. Ensure datetime format
df['post_date'] = pd.to_datetime(df['post_date'])

# 2. Limit to last 2 full months
latest_date = df['post_date'].max()
end_of_last_month = latest_date.replace(day=1)
start_of_two_months_ago = end_of_last_month - pd.DateOffset(months=2)

df_recent = df[df['post_date'] >= start_of_two_months_ago].copy()

# 3. Add month column
df_recent['month'] = df_recent['post_date'].dt.to_period('M')

# 4. Count hashtags per month
hashtag_counts = df_recent.groupby(['hashtag_name', 'month']).size().unstack(fill_value=0)

# 5. Get last two months
prev_month = hashtag_counts.columns[-2]
last_month = hashtag_counts.columns[-1]

# 6. Filter: ≥10 mentions before, ≥50 now, and growing
filtered = hashtag_counts[
    (hashtag_counts[prev_month] >= 10) &
    (hashtag_counts[last_month] >= 5) &
    (hashtag_counts[last_month] > hashtag_counts[prev_month])
].copy()

# 7. Calculate growth rate
filtered['growth_rate'] = (
    (filtered[last_month] - filtered[prev_month]) / 
    filtered[prev_month].replace(0, 1)
)

# 8. Get top 20 by growth
top_hashtags_df = filtered.sort_values('growth_rate', ascending=False).head(20)
top_hashtags = top_hashtags_df.index.tolist()

# 9. Get example posts for those hashtags from the last month
example_posts = (
    df[
        (df['hashtag_name'].isin(top_hashtags)) &
        (df['post_date'].dt.to_period('M') == last_month)
    ]
    .sort_values('diggCount', ascending=False)
    .groupby('hashtag_name')
    .agg({
        'text': 'first',
        'webVideoUrl': 'first',
        'diggCount': 'first'
    })
)

# 10. Join example posts to top hashtags
final = top_hashtags_df.join(example_posts)

# 11. Normalize "Interest over time" per hashtag (peak = 100)
# Compute from the full `hashtag_counts`, not just the 2-month filtered dataset
interest_over_time = (
    hashtag_counts
    .div(hashtag_counts.max(axis=1), axis=0)
    * 100
).round(1)

# Then filter for top hashtags
interest_over_time = interest_over_time.loc[top_hashtags]


# 12. Merge interest over time into final DataFrame
final = final.join(interest_over_time, rsuffix='_interest')




In [33]:
# Convert Periods to strings
prev_month_str = str(prev_month)
last_month_str = str(last_month)

# Define renamed column names
prev_col = f'mentions_{prev_month_str}'
last_col = f'mentions_{last_month_str}'

# Rename columns (use string versions)
final = final.rename(columns={
    prev_month_str: prev_col,
    last_month_str: last_col,
    'text': 'example_caption',
    'webVideoUrl': 'video_link',
    'diggCount': 'likes'
})

result_df = final.reset_index()[[
    'hashtag_name',
    prev_col,
    last_col,
    'growth_rate',
    'example_caption',
    'video_link',
    f"{prev_month_str}_interest",
    f"{last_month_str}_interest"
]]



In [34]:
final

,2025-04,mentions_2025-05,mentions_2025-06,growth_rate,example_caption,video_link,likes,2025-04_interest,2025-05_interest,2025-06_interest
hashtag_name,,,,,,,,,,
tiktokshop,10,10,19,0.900000,Ich liebe diese Glas Skin Maske von @svenja.walberg 🩷 schon beim ersten Mal hat die skincare Maske mich sehr überzeugt! Sichert euch jetzt die Maske auf #tiktokshopdeutschland #tiktokmademebuyit #tiktokshop #skincare #maske #svenjawalberg #hautpflege,https://www.tiktok.com/@melisabaniz/video/7512192631254945046,5190,52.6,52.6,100.0
melanin,0,11,20,0.818182,"🎯 Vitiligo? Hol dir deine Haut zurück – natürlich. Weißflecken satt? Die Fivfivgo® VitiCalm Bienengift-Creme ist dein Gamechanger! 💥 Dermatologisch bestätigt, sanft & effektiv – ganz ohne aggressive Behandlungen. 🐝 Mit Bienenvenom + pflanzlichen Wirkstoffen ✅ Regt die Melaninproduktion an (ohne Kortison!) ✅ Beruhigt das Immunsystem ✅ Sichtbare Ergebnisse in 7–10 Tagen ✅ Spendet Feuchtigkeit & gleicht den Hautton aus Tausende Nutzer feiern ihre Haut-Rückkehr. 📲 Jetzt tippen & deine Hautreise starten! #Vitiligo#Melanin#Hautpflege#NatürlichHeilen",https://www.tiktok.com/@peachyxchic_official/video/7512315428853533974,254,0.0,55.0,100.0
drogerie,10,10,15,0.500000,"Ich kann das alles wirklich nicht mehr, gefärbte vaseline für 19€, ich drehe durch 😭 #skincare #hautpflege #drogerie #fyp #viral",https://www.tiktok.com/@xskincare/video/7513510848074108182,45300,66.7,66.7,100.0
hautgesundheit,7,23,34,0.478261,"🌊💧 Warum spritziges, kohlensäurehaltiges Wasser nicht die beste Wahl für deine Haut ist! 💧🌊 🌟 Heute wollen wir über etwas sprechen, das vielen von uns am Herzen liegt: unsere Haut, insbesondere bei Akne. Viele von uns greifen gerne zu spritzigem, kohlensäurehaltigem Wasser – erfrischend und lecker, oder? Aber ist es wirklich so gut für unsere Haut? 👉 Hier sind einige Gründe, warum kohlensäurehaltiges Wasser nicht ideal für Akne sein könnte: 1. Säuregehalt: Kohlensäurehaltiges Wasser hat einen niedrigeren pH-Wert und ist damit saurer. Ein zu saurer pH-Wert kann die Hautbarriere angreifen und zu Reizungen führen, besonders bei empfindlicher Haut. 2. Entzündungen: Die enthaltenen Kohlensäureblasen können bei manchen Menschen ein Hautempfinden von „Prickeln“ oder „Stechen“ auslösen, das sich negativ auf entzündliche Hautzustände wie Akne auswirken kann. 3. Dehydrierung: Obwohl spritziges Wasser erfrischend sein kann, kann der hohe Kohlensäuregehalt dazu führen, dass einige Menschen weniger durstig sind und dadurch die empfohlene Tagesmenge an Flüssigkeit nicht erreichen. Eine gute Hydration ist jedoch entscheidend für eine gesunde Haut! 💡 Tipps für deine Hautpflege: Anstatt zu kohlensäurehaltigem Wasser zu greifen, versuche es mit stillem Wasser oder Tees! Diese Optionen sind nicht nur sanfter zu deiner Haut, sondern unterstützen auch die optimale Hydratation. Was trinkt ihr am liebsten? Lasst es uns in den Kommentaren wissen! 👇💬 Bleibt gesund und sorgt gut für eure Haut! 🌟💖 #Hautpflege #Akne #Kohlensäure #Hautgesundheit #StayHydrated #BeautyTips Aknebehandlung, Hautpflege, Kohlensäurehaltiges Wasser, Sprudelwasser und Hautgesundheit, Hautirritation, pH-Wert der Haut, Hautpflege-Tipps, Kohlenstoffdioxid und Haut, Säure in Getränken, Hautfeuchtigkeit, Stilles Wasser vs. Sprudelwasser, Entzündungshemmende Maßnahmen, Gesunde Haut, Hauttyp und Ernährung, Hydratation und Akne, Kohlensäure und Hautempfindlichkeit, Ernährung für klare Haut, Natürliche Hautpflege, Kohlensäurehaltige Getränke und Gesundheit, Akne und Ernährung.",https://www.tiktok.com/@medicalcosmeticnina/video/7512770112462654743,13500,20.6,67.6,100.0
fypシ,20,17,23,0.352941,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49

In [55]:
import pandas as pd
from datetime import timedelta

# STEP 1: Load data
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])

# STEP 2: Create post_date
df['post_date'] = df['createTimeISO'].dt.date
df['hashtag_name'] = df['hashtag_name'].str.lower().str.strip()

# STEP 3: Define date windows
today = df['post_date'].max()
this_week_start = today - timedelta(days=6)
last_week_start = this_week_start - timedelta(days=7)
week_before_last_start = last_week_start - timedelta(days=7)

# STEP 4: Weekly slices
df_this_week = df[df['post_date'] >= this_week_start]
df_last_week = df[(df['post_date'] >= last_week_start) & (df['post_date'] < this_week_start)]
df_week_before_last = df[(df['post_date'] >= week_before_last_start) & (df['post_date'] < last_week_start)]

# STEP 5: Unique (post_id, hashtag_name) pairs
df_this_unique = df_this_week[['post_id', 'hashtag_name', 'post_date']].drop_duplicates()
df_last_unique = df_last_week[['post_id', 'hashtag_name', 'post_date']].drop_duplicates()
df_week_before_last_unique = df_week_before_last[['post_id', 'hashtag_name', 'post_date']].drop_duplicates()

# STEP 6: Interest over time
daily_counts = (
    df_this_unique.groupby(['hashtag_name', 'post_date'])
    .size()
    .reset_index(name='count')
)
daily_counts['max'] = daily_counts.groupby('hashtag_name')['count'].transform('max')
daily_counts['interest'] = (daily_counts['count'] / daily_counts['max'] * 100).round(1)
daily_counts.drop(columns='max', inplace=True)

# STEP 7: Weekly counts
this_week_counts = df_this_unique.groupby('hashtag_name').size().reset_index(name='this_week_mentions')
last_week_counts = df_last_unique.groupby('hashtag_name').size().reset_index(name='last_week_mentions')
week_before_last_counts = df_week_before_last_unique.groupby('hashtag_name').size().reset_index(name='week_before_last_mentions')

# STEP 8: Merge trends
hashtag_trend = pd.merge(this_week_counts, last_week_counts, on='hashtag_name', how='outer')
hashtag_trend = pd.merge(hashtag_trend, week_before_last_counts, on='hashtag_name', how='outer')
hashtag_trend.fillna(0, inplace=True)

# STEP 9: Trend calculations
hashtag_trend['trend_change_pct'] = (
    (hashtag_trend['this_week_mentions'] - hashtag_trend['last_week_mentions']) /
    hashtag_trend['last_week_mentions'].replace(0, 1)
).round(2)

hashtag_trend['last_week_trend_pct'] = (
    (hashtag_trend['last_week_mentions'] - hashtag_trend['week_before_last_mentions']) /
    hashtag_trend['week_before_last_mentions'].replace(0, 1)
).round(2)

def label_change(pct):
    if pct > 0.5:
        return "📈 Rising"
    elif pct < -0.5:
        return "📉 Dropping"
    else:
        return "⏸️ Stable"

hashtag_trend['trend_label'] = hashtag_trend['trend_change_pct'].apply(label_change)
hashtag_trend['last_week_trend_label'] = hashtag_trend['last_week_trend_pct'].apply(label_change)

# STEP 10: Top hashtags
top_tags = hashtag_trend.sort_values('this_week_mentions', ascending=False).head(20)['hashtag_name'].tolist()

# STEP 11: Example posts
example_posts = (
    df_this_week[df_this_week['hashtag_name'].isin(top_tags)]
    .sort_values('diggCount', ascending=False)
    .groupby('hashtag_name')
    .agg({
        'text': 'first',
        'webVideoUrl': 'first',
        'diggCount': 'first',
        'post_date': 'first'
    })
    .reset_index()
)

# STEP 12: Merge and display
final_df = pd.merge(example_posts, hashtag_trend, on='hashtag_name', how='left')
final_df = final_df.sort_values('this_week_mentions', ascending=False)

pd.set_option('display.max_colwidth', None)
final_df_display = final_df[[
    'hashtag_name',
    'post_date',
    'diggCount',
    'this_week_mentions',
    'last_week_mentions',
    'week_before_last_mentions',
    'trend_change_pct',
    'trend_label',
    'last_week_trend_pct',
    'last_week_trend_label',
    'text',
    'webVideoUrl'
]]



In [56]:
final_df

,hashtag_name,text,webVideoUrl,diggCount,post_date,this_week_mentions,last_week_mentions,week_before_last_mentions,trend_change_pct,last_week_trend_pct,trend_label,last_week_trend_label
15,skincare,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950,171600,2025-06-22,181.0,188.0,171.0,-0.04,0.10,⏸️ Stable,⏸️ Stable
16,skincareroutine,summer bodycare routine #skincare #bodycare #morningroutine #skincareroutine #reaperdame,https://www.tiktok.com/@reaperdame/video/7517828509608217869,108700,2025-06-20,114.0,125.0,122.0,-0.09,0.02,⏸️ Stable,⏸️ Stable
4,fyp,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,65.0,77.0,85.0,-0.16,-0.09,⏸️ Stable,⏸️ Stable
8,hautpflege,Anzeige| 50 Meter über dem Boden – aber UV-Schutz bleibt Pflicht ☀️💛 Anthelios UV Air Fluid LSF 50+ immer dabei. @La Roche-Posay #larocheposayskincare #hautpflege #sonnenschutz #larocheposay #Anthelios #UVAir #The77Shift #Suncare,https://www.tiktok.com/@medsri/video/7518079530649554198,12000,2025-06-20,65.0,108.0,145.0,-0.40,-0.26,⏸️ Stable,⏸️ Stable
10,koreanskincare,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,45.0,41.0,53.0,0.10,-0.23,⏸️ Stable,⏸️ Stable
17,skincaretips,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,28.0,30.0,31.0,-0.07,-0.03,⏸️ Stable,⏸️ Stable
7,grwm,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950,171600,2025-06-22,28.0,30.0,20.0,-0.07,0.50,⏸️ Stable,⏸️ Stable
9,kbeauty,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,27.0,31.0,33.0,-0.13,-0.06,⏸️ Stable,⏸️ Stable
5,glassskin,I am not going to let my hard work go to waste!! Neck routine on plane 🌚 retinAl is my favorite #celimax #retinal #koreanskincare #glassskin #glowyskin #koreanbeauty #neckroutine #techneck #skincare101 #faceyoga,https://www.tiktok.com/@songofskin/video/75188104

In [45]:
# STEP 10: Plot interest-over-time for top hashtags
def plot_hashtag_trend_7d(tag):
    subset = daily_counts[daily_counts['hashtag_name'] == tag]
    if subset.empty:
        return
    plt.plot(subset['post_date'], subset['interest'], marker='o', label=f'#{tag}')
    plt.title(f'7-Day Interest: #{tag}')
    plt.ylim(0, 100)
    plt.xlabel('Date')
    plt.ylabel('Interest (0–100)')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

#for tag in top_tags:
 #   plot_hashtag_trend_7d(tag)


In [57]:
# Exclude specific hashtags just before final display
excluded_tags = ['skincare', 'skincareroutine', 'hautpflege', 'hautpflegeroutine']
filtered_final_df = final_df[~final_df['hashtag_name'].str.lower().isin(excluded_tags)]


In [58]:
filtered_final_df

,hashtag_name,text,webVideoUrl,diggCount,post_date,this_week_mentions,last_week_mentions,week_before_last_mentions,trend_change_pct,last_week_trend_pct,trend_label,last_week_trend_label
4,fyp,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,65.0,77.0,85.0,-0.16,-0.09,⏸️ Stable,⏸️ Stable
10,koreanskincare,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,45.0,41.0,53.0,0.10,-0.23,⏸️ Stable,⏸️ Stable
17,skincaretips,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,28.0,30.0,31.0,-0.07,-0.03,⏸️ Stable,⏸️ Stable
7,grwm,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950,171600,2025-06-22,28.0,30.0,20.0,-0.07,0.50,⏸️ Stable,⏸️ Stable
9,kbeauty,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,27.0,31.0,33.0,-0.13,-0.06,⏸️ Stable,⏸️ Stable
5,glassskin,I am not going to let my hard work go to waste!! Neck routine on plane 🌚 retinAl is my favorite #celimax #retinal #koreanskincare #glassskin #glowyskin #koreanbeauty #neckroutine #techneck #skincare101 #faceyoga,https://www.tiktok.com/@songofskin/video/7518810411282517262,58500,2025-06-22,25.0,16.0,20.0,0.56,-0.20,📈 Rising,⏸️ Stable
13,selfcare,Guess my favorite color yet?🫧🫧🫧\n#asmr #asmrroutine #morningroutine #skincare #selfcare,https://www.tiktok.com/@butenk_nataliia/video/7518364640934399237,27300,2025-06-21,23.0,38.0,30.0,-0.39,0.27,⏸️ Stable,⏸️ Stable
6,glowup,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,2025-06-24,20.0,26.0,30.0,-0.23,-0.13,⏸️ Stable,⏸️ Stable
12,routine,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486,754600,20

In [ ]:
import pandas as pd
from datetime import timedelta

# Load uploaded dataset
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
df['post_date'] = df['createTimeISO'].dt.date
df['hashtag_name'] = df['hashtag_name'].str.lower().str.strip()

# PART 1: Z-SCORE TREND
latest_date = df['post_date'].max()
start_date = latest_date - timedelta(days=27)
df_recent = df[df['post_date'] >= start_date]

daily_counts = (
    df_recent[['post_id', 'hashtag_name', 'post_date']].drop_duplicates()
    .groupby(['hashtag_name', 'post_date']).size().reset_index(name='count')
)

hashtag_stats = (
    daily_counts.groupby('hashtag_name')['count']
    .agg(['mean', 'std']).reset_index()
)
daily_counts = pd.merge(daily_counts, hashtag_stats, on='hashtag_name', how='left')
daily_counts['z_score'] = ((daily_counts['count'] - daily_counts['mean']) / daily_counts['std'].replace(0, 1)).round(2)

latest_z = daily_counts[daily_counts['post_date'] == latest_date].sort_values('z_score', ascending=False)
latest_z_df = latest_z[['hashtag_name', 'count', 'mean', 'z_score']].head(20)

# PART 2: RELATIVE SHARE TREND
this_week_start = latest_date - timedelta(days=6)
last_week_start = this_week_start - timedelta(days=7)

this_week = df[df['post_date'] >= this_week_start]
last_week = df[(df['post_date'] >= last_week_start) & (df['post_date'] < this_week_start)]

this_week_unique = this_week[['post_id', 'hashtag_name']].drop_duplicates()
last_week_unique = last_week[['post_id', 'hashtag_name']].drop_duplicates()

tw = this_week_unique.groupby('hashtag_name').size().reset_index(name='this_week_mentions')
lw = last_week_unique.groupby('hashtag_name').size().reset_index(name='last_week_mentions')

merged = pd.merge(tw, lw, on='hashtag_name', how='outer').fillna(0)
total_tw = merged['this_week_mentions'].sum()
total_lw = merged['last_week_mentions'].sum()

merged['this_week_share'] = merged['this_week_mentions'] / total_tw
merged['last_week_share'] = merged['last_week_mentions'] / total_lw

merged['share_change_pct'] = (
    (merged['this_week_share'] - merged['last_week_share']) /
    merged['last_week_share'].replace(0, 1)
).round(2)

def trend_label(pct):
    if pct > 0.1:
        return "📈 Rising"
    elif pct < -0.1:
        return "📉 Dropping"
    else:
        return "⏸️ Stable"

merged['trend_label'] = merged['share_change_pct'].apply(trend_label)
top_rel_df = merged.sort_values('this_week_share', ascending=False).head(20)
top_rel_df = top_rel_df[['hashtag_name', 'this_week_mentions', 'last_week_mentions', 'share_change_pct', 'trend_label']]



(               hashtag_name  count      mean  z_score
 7663       skincareobsessed      1  1.000000     0.00
 6442           poreobsessed      1  1.000000     0.00
 5986           nofilterskin      1  1.000000     0.00
 8008             skintokfav      1  1.000000     0.00
 4545         kbeautyroutine      1  1.000000     0.00
 8253              stylevana      1  1.000000     0.00
 6269         pflegeprodukte      1  1.000000     0.00
 5467                momlife      1  1.000000     0.00
 8348        svbeautifulself      1  1.000000     0.00
 8543          tiktokgermany      1  1.000000     0.00
 2827                frische      1  1.000000     0.00
 6825                ringana      1  1.000000     0.00
 7247  sensitiveskinfriendly      1  1.000000     0.00
 8976             vanafamily      1  1.000000     0.00
 7976               skintips      1  1.000000     0.00
 3909              hautliebe      1  1.090909    -0.30
 3477          glowuproutine      1  1.100000    -0.32
 7705     

In [64]:
latest_z_df


,hashtag_name,count,mean,z_score
7663,skincareobsessed,1,1.000000,0.00
6442,poreobsessed,1,1.000000,0.00
5986,nofilterskin,1,1.000000,0.00
8008,skintokfav,1,1.000000,0.00
4545,kbeautyroutine,1,1.000000,0.00
8253,stylevana,1,1.000000,0.00
6269,pflegeprodukte,1,1.000000,0.00
5467,momlife,1,1.000000,0.00
8348,svbeautifulself,1,1.000000,0.00
8543,tiktokgermany,1,1.000000,0.00


In [63]:
 top_rel_df

,hashtag_name,this_week_mentions,last_week_mentions,share_change_pct,trend_label
2188,skincare,181.0,188.0,0.34,📈 Rising
2251,skincareroutine,114.0,125.0,0.27,📈 Rising
1138,hautpflege,65.0,108.0,-0.16,📉 Dropping
913,fyp,65.0,77.0,0.18,📈 Rising
1371,koreanskincare,45.0,41.0,0.53,📈 Rising
1064,grwm,28.0,30.0,0.30,📈 Rising
2267,skincaretips,28.0,30.0,0.30,📈 Rising
1324,kbeauty,27.0,31.0,0.21,📈 Rising
1004,glassskin,25.0,16.0,1.18,📈 Rising
2115,selfcare,23.0,38.0,-0.16,📉 Dropping


In [74]:
import pandas as pd
from datetime import timedelta

# Load and preprocess
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
df['post_date'] = df['createTimeISO'].dt.date
df['week'] = pd.to_datetime(df['post_date']).dt.to_period('W').apply(lambda r: r.start_time)
df['hashtag_name'] = df['hashtag_name'].str.lower().str.strip()

# Select the last 6 full weeks (to calculate 2 rolling z-scores)
latest_6_weeks = sorted(df['week'].unique())[-6:]
df_recent = df[df['week'].isin(latest_6_weeks)]

# Weekly unique hashtag counts
weekly_counts = (
    df_recent[['post_id', 'hashtag_name', 'week']].drop_duplicates()
    .groupby(['hashtag_name', 'week']).size().reset_index(name='count')
)

# Pivot for easier windowed access
pivot = weekly_counts.pivot(index='hashtag_name', columns='week', values='count').fillna(0)


# Get the 6 weeks in order
w1, w2, w3, w4, w5, w6 = latest_6_weeks  # oldest to newest


# Optional noise filter: remove hashtags with low total volume in 6 weeks
pivot['total_mentions'] = pivot[[w1, w2, w3, w4, w5, w6]].sum(axis=1)
pivot = pivot[pivot['total_mentions'] >= 5]

# Calculate means and stds
pivot['last_week_mean'] = pivot[[w1, w2, w3, w4]].mean(axis=1)
pivot['last_week_std'] = pivot[[w1, w2, w3, w4]].std(axis=1).replace(0, 1)
pivot['z_score_last_week'] = ((pivot[w5] - pivot['last_week_mean']) / pivot['last_week_std']).round(2)

pivot['this_week_mean'] = pivot[[w2, w3, w4, w5]].mean(axis=1)
pivot['this_week_std'] = pivot[[w2, w3, w4, w5]].std(axis=1).replace(0, 1)
pivot['z_score_this_week'] = ((pivot[w6] - pivot['this_week_mean']) / pivot['this_week_std']).round(2)

# Compare Z-scores
pivot['z_score_change'] = (pivot['z_score_this_week'] - pivot['z_score_last_week']).round(2)

# Result
result = pivot.reset_index().sort_values('z_score_this_week', ascending=False)[[
    'hashtag_name',
    w5, 'z_score_last_week',
    w6, 'z_score_this_week',
    'z_score_change'
]].head(20)
result = result.sort_values('z_score_this_week', ascending=False)
result

week,hashtag_name,2025-06-16 00:00:00,z_score_last_week,2025-06-23 00:00:00,z_score_this_week,z_score_change
390,panthenol,1.0,0.87,3.0,4.50,3.63
345,microinfluencer,0.0,-1.22,3.0,4.33,5.55
78,blackgirlskincare,1.0,0.50,2.0,3.50,3.00
338,medicubedevice,0.0,-0.78,2.0,3.50,4.28
499,skincareroutinetips,1.0,-0.71,6.0,3.49,4.20
95,celimax,3.0,0.87,4.0,2.60,1.73
519,sommer,0.0,-1.50,2.0,2.60,4.10
48,beautycontent,0.0,-1.00,2.0,2.50,3.50
432,rossmann,1.0,0.50,2.0,2.50,2.00
228,goviral,2.0,0.87,3.0,2.50,1.63


In [76]:
# Load and preprocess (skip if already done)
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])
df['post_date'] = df['createTimeISO'].dt.date
df['week'] = pd.to_datetime(df['post_date']).dt.to_period('W').apply(lambda r: r.start_time)
df['hashtag_name'] = df['hashtag_name'].str.lower().str.strip()

# Use the same 6-week window as z-score
latest_6_weeks = sorted(df['week'].unique())[-6:]
df_recent = df[df['week'].isin(latest_6_weeks)]

# Count unique posts per hashtag per week
weekly_counts = (
    df_recent[['post_id', 'hashtag_name', 'week']].drop_duplicates()
    .groupby(['hashtag_name', 'week']).size().reset_index(name='mentions')
)

# Total posts per week (denominator for relative share)
weekly_totals = (
    df_recent[['post_id', 'week']].drop_duplicates()
    .groupby('week').size().reset_index(name='total_posts')
)

# Merge and calculate share per week
df_share = pd.merge(weekly_counts, weekly_totals, on='week', how='left')
df_share['share'] = df_share['mentions'] / df_share['total_posts']

# Pivot for comparison
pivot = df_share.pivot(index='hashtag_name', columns='week', values='share').fillna(0)


# Get weeks
w1, w2, w3, w4, w5, w6 = latest_6_weeks  # oldest to newest
# Add back raw counts for w5 and w6 to filter noise
raw_counts = weekly_counts.pivot(index='hashtag_name', columns='week', values='mentions').fillna(0)
pivot['mentions_last_week'] = raw_counts.get(w5, 0)
pivot['mentions_this_week'] = raw_counts.get(w6, 0)

# Only keep hashtags with at least 5 posts in either week
pivot = pivot[(pivot['mentions_last_week'] >= 5) | (pivot['mentions_this_week'] >= 5)]


# Compare this week (w6) vs. last week (w5)
pivot['share_change_pct'] = ((pivot[w6] - pivot[w5]) / pivot[w5].replace(0, 1)).round(2)

def label(pct):
    if pct > 0.1:
        return "Rising"
    elif pct < -0.1:
        return "Dropping"
    else:
        return "Stable"

pivot['trend_label'] = pivot['share_change_pct'].apply(label)

# Final result
share_result = pivot.reset_index().sort_values(w6, ascending=False).head(20)
share_result = share_result[[
    'hashtag_name',
    w5, w6,
    'share_change_pct',
    'trend_label'
]]

share_result


week,hashtag_name,2025-06-16 00:00:00,2025-06-23 00:00:00,share_change_pct,trend_label
67,skincare,0.468619,0.447761,-0.04,Stable
72,skincareroutine,0.282427,0.303483,0.07,Stable
41,hautpflege,0.196653,0.194030,-0.01,Stable
23,fyp,0.173640,0.169154,-0.03,Stable
47,koreanskincare,0.100418,0.114428,0.14,Rising
45,kbeauty,0.046025,0.089552,0.95,Rising
75,skincaretips,0.062762,0.084577,0.35,Rising
31,glassskin,0.041841,0.074627,0.78,Rising
35,grwm,0.087866,0.054726,-0.38,Dropping
33,glowup,0.060669,0.049751,-0.18,Dropping


In [82]:
import pandas as pd
from datetime import timedelta

# STEP 1: Load and preprocess
df = pd.read_csv("/Users/ritushetkar/env_capstone/data/hashtags_posts.csv", parse_dates=['createTimeISO'])  # Replace with actual file path
df['post_date'] = df['createTimeISO'].dt.date
df['hashtag_name'] = df['hashtag_name'].str.lower().str.strip()

# STEP 2: Define date windows
today = df['post_date'].max()
this_week_start = today - timedelta(days=6)
last_week_start = this_week_start - timedelta(days=7)
week_before_last_start = last_week_start - timedelta(days=7)

# STEP 3: Weekly slices
df_this_week = df[df['post_date'] >= this_week_start]
df_last_week = df[(df['post_date'] >= last_week_start) & (df['post_date'] < this_week_start)]
df_week_before_last = df[(df['post_date'] >= week_before_last_start) & (df['post_date'] < last_week_start)]

# STEP 4: Unique (post_id, hashtag_name) pairs
def get_hashtag_share(df_week):
    unique = df_week[['post_id', 'hashtag_name']].drop_duplicates()
    counts = unique.groupby('hashtag_name').size().reset_index(name='mentions')
    total_posts = unique['post_id'].nunique()
    counts['share'] = counts['mentions'] / total_posts
    return counts.set_index('hashtag_name')[['share']]

this_week_share = get_hashtag_share(df_this_week).rename(columns={'share': 'this_week_share'})
last_week_share = get_hashtag_share(df_last_week).rename(columns={'share': 'last_week_share'})
week_before_last_share = get_hashtag_share(df_week_before_last).rename(columns={'share': 'week_before_last_share'})

# STEP 5: Merge share data
trend_df = this_week_share.join(last_week_share, how='outer').join(week_before_last_share, how='outer').fillna(0)

# STEP 6: Calculate relative change and label
trend_df['trend_change_pct'] = ((trend_df['this_week_share'] - trend_df['last_week_share']) / trend_df['last_week_share'].replace(0, 1)).round(2)
trend_df['last_week_trend_pct'] = ((trend_df['last_week_share'] - trend_df['week_before_last_share']) / trend_df['week_before_last_share'].replace(0, 1)).round(2)

q1 = trend_df['trend_change_pct'].quantile(0.25)
q3 = trend_df['trend_change_pct'].quantile(0.75)
iqr = q3 - q1

upper_threshold = q3 + 1.5 * iqr
lower_threshold = q1 - 1.5 * iqr

def label_change(pct):
    if pct > upper_threshold:
        return "📈 Rising"
    elif pct < lower_threshold:
        return "📉 Dropping"
    else:
        return "⏸️ Stable"




trend_df['trend_label'] = trend_df['trend_change_pct'].apply(label_change)
trend_df['last_week_trend_label'] = trend_df['last_week_trend_pct'].apply(label_change)

# STEP 7: Get top hashtags
top_tags = trend_df.sort_values('this_week_share', ascending=False).head(20).index.tolist()

# STEP 8: Example posts
example_posts = (
    df_this_week[df_this_week['hashtag_name'].isin(top_tags)]
    .sort_values('diggCount', ascending=False)
    .groupby('hashtag_name')
    .agg({
        'text': 'first',
        'webVideoUrl': 'first',
        'diggCount': 'first',
        'post_date': 'first'
    })
)

# STEP 9: Merge and output
final_df = example_posts.join(trend_df, how='left').sort_values('this_week_share', ascending=False)
final_df_display = final_df.reset_index()[[
    'hashtag_name',
    'post_date',
    'diggCount',
    'this_week_share',
    'last_week_share',
    'week_before_last_share',
    'trend_change_pct',
    'trend_label',
    'last_week_trend_pct',
    'last_week_trend_label',
    'text',
    'webVideoUrl'
]]

final_df_display


,hashtag_name,post_date,diggCount,this_week_share,last_week_share,week_before_last_share,trend_change_pct,trend_label,last_week_trend_pct,last_week_trend_label,text,webVideoUrl
0,skincare,2025-06-22,171600,0.480106,0.411379,0.361522,0.17,⏸️ Stable,0.14,⏸️ Stable,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950
1,skincareroutine,2025-06-20,108700,0.302387,0.273523,0.257928,0.11,⏸️ Stable,0.06,⏸️ Stable,summer bodycare routine #skincare #bodycare #morningroutine #skincareroutine #reaperdame,https://www.tiktok.com/@reaperdame/video/7517828509608217869
2,fyp,2025-06-24,754600,0.172414,0.168490,0.179704,0.02,⏸️ Stable,-0.06,⏸️ Stable,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486
3,hautpflege,2025-06-20,12000,0.172414,0.236324,0.306554,-0.27,⏸️ Stable,-0.23,⏸️ Stable,Anzeige| 50 Meter über dem Boden – aber UV-Schutz bleibt Pflicht ☀️💛 Anthelios UV Air Fluid LSF 50+ immer dabei. @La Roche-Posay #larocheposayskincare #hautpflege #sonnenschutz #larocheposay #Anthelios #UVAir #The77Shift #Suncare,https://www.tiktok.com/@medsri/video/7518079530649554198
4,koreanskincare,2025-06-24,754600,0.119363,0.089716,0.112051,0.33,⏸️ Stable,-0.20,⏸️ Stable,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486
5,skincaretips,2025-06-24,754600,0.074271,0.065646,0.065539,0.13,⏸️ Stable,0.00,⏸️ Stable,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486
6,grwm,2025-06-22,171600,0.074271,0.065646,0.042283,0.13,⏸️ Stable,0.55,⏸️ Stable,GRWM & JASPER TO GO TO THE BEACH! 🏖️ | #fypシ #grwm #foryoupage #grwmroutine #fypシ゚viral #viralvideo #skincare #koreanskincare #darkspot #hyperpigmentation #amazon #ulta #spf #whitecast #darkspottreatment #viral @Anua Store US \nStore link!\nhttps://www.amazon.com/Niacinamide-Tranexamic-Hyaluronic-Sensitive-Fragrance-Free/dp/B0CLLV2T1P/?maas=maas_adg_9BCC1D0835352D49B9FDB92722B84F8C_afap_abs&ref_=aa_maas&tag=maas,https://www.tiktok.com/@alyforlife2/video/7518657191629425950
7,kbeauty,2025-06-24,754600,0.071618,0.067834,0.069767,0.06,⏸️ Stable,-0.03,⏸️ Stable,"Tub time = stress? Gone. Skin care? No one’s doing it for you, babe. Gotta do it myself❤️ 🧊All Skin care : @reiireii.official #routine #nightroutine #glowup #kbeauty #skincaretips #tiktokviral #trending #koreanskin #koreanskincare #routine #routines #korean #glowup #koreatrip #life #nightroutine #fyp #foryourpage",https://www.tiktok.com/@korea_grandma/video/7519490156294835486
8,glassskin,2025-06-22,58500,0.066313,0.035011,0.042283,0.89,⏸️ Stable,-0.17,⏸️ Stable,I am not going to let my hard work go to waste!! Neck routine on plane 🌚 retinAl is my favorite #celimax #retinal #kor